# 03: Deploy Model on Vertex AI

This notebook demonstrates how to deploy a TensorFlow SavedModel from Google Cloud Storage (GCS) to Vertex AI for serving.

## Prerequisites

- You have uploaded your model to GCS (e.g. `gs://jinhomlee-movenet-2024/movenet/saved_model/`).
- Vertex AI and the Vertex AI API are enabled in your Google Cloud project.
- You have installed `google-cloud-aiplatform` in your environment.
- Your environment is authenticated to use GCP (see [Google ADC Troubleshooting](https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds)).

In [1]:
# Install the Vertex AI SDK if not already installed
!pip install -U google-cloud-aiplatform

  Using cached protobuf-6.32.0-cp39-abi3-macosx_10_9_universal2.whl (426 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.21 requires protobuf<5,>=4.25.3, but you have protobuf 6.32.0 which is incompatible.
tensorflow 2.19.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.32.0 which is incompatible.


In [2]:
from google.cloud import aiplatform
from google.cloud.aiplatform import gapic as aiplatform_gapic
import os
import config  # import your config file

## Set up Vertex AI project and location

In [3]:
# Update these variables for your setup
PROJECT_ID = config.PROJECT_ID
REGION = config.REGION
MODEL_DISPLAY_NAME = config.MODEL_DISPLAY_NAME
GCS_MODEL_URI = config.GCS_MODEL_URI  # Use this instead of GCS_MODEL_PATH for full URI

In [4]:
# Authenticate (if running outside GCP, e.g. Colab)
# from google.colab import auth
# auth.authenticate_user()

In [5]:
# Initialize Vertex AI
aiplatform.init(project=PROJECT_ID, location=REGION)

## Upload the model to Vertex AI Model Registry

The following code does what can be also achieved by using the "Import" button (right of "+Create") at the top of https://console.cloud.google.com/vertex-ai/models?project=shuttle-insights: 

![Vertex AI Model Registry Import](../images/vertex-ai-model-registry-import.png)

In [6]:
model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=GCS_MODEL_URI,
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-11:latest"  # Or tf2-gpu if needed
)
print(f"Model resource name: {model.resource_name}")

/Users/jin-holee/github/Jin-HoMLee/shuttle-insights/.venv/lib/python3.11/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Creating Model
Create Model backing LRO: projects/495366704424/locations/us-central1/models/8439476321343504384/operations/7645746041257459712
Model created. Resource name: projects/495366704424/locations/us-central1/models/8439476321343504384@1
To use this Model in another session:
model = aiplatform.Model('projects/495366704424/locations/us-central1/models/8439476321343504384@1')
Model resource name: projects/495366704424/locations/us-central1/models/8439476321343504384


## Deploy the model to an endpoint

The following code cells deploy the model to an endpoint. The same can be achieved on https://console.cloud.google.com/vertex-ai/models?project=shuttle-insights by clicking on the three dots right-most of the model and then "Deploy to endpoint". 
Under "Define your endpoint", the "Endpoint name" is set to e.g. "movenet-endpoint" and all other settings are kept as default. 
Under "Model settings", the "Machine type" is set to e.g. "n1-standard-2". 

![Vertex AI Model Registry Deploy to Endpoint](../images/vertex-ai-model-registry-deploy.png)



In [7]:
# Create an endpoint for online prediction
endpoint = aiplatform.Endpoint.create(display_name="movenet-endpoint")
print(f"Endpoint resource name: {endpoint.resource_name}")

Creating Endpoint
Create Endpoint backing LRO: projects/495366704424/locations/us-central1/endpoints/6225420542816550912/operations/269412801578008576
Endpoint created. Resource name: projects/495366704424/locations/us-central1/endpoints/6225420542816550912
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/495366704424/locations/us-central1/endpoints/6225420542816550912')
Endpoint resource name: projects/495366704424/locations/us-central1/endpoints/6225420542816550912


In [8]:
# Deploy the model to the endpoint
deployed_model = model.deploy(
    endpoint=endpoint,
    machine_type="n1-standard-2"  # You can change the machine type
)

Deploying model to Endpoint : projects/495366704424/locations/us-central1/endpoints/6225420542816550912
Deploy Endpoint model backing LRO: projects/495366704424/locations/us-central1/endpoints/6225420542816550912/operations/7687967287764058112
Endpoint model deployed. Resource name: projects/495366704424/locations/us-central1/endpoints/6225420542816550912


In [9]:
# Print deployed model info
for dm in endpoint.list_models():
    print(f"Deployed model ID: {dm.id}, display name: {dm.display_name}")
    print(f"Endpoint ID: {endpoint.name}")


Deployed model ID: 561185237258731520, display name: movenet-tf
Endpoint ID: 6225420542816550912


## Undeploy Model (whenever not using as you pay for resources) 

E.g. at the end of a day. 

Here’s a rough cost estimate:

- Endpoint (n1-standard-2):
    - About $0.10 per hour (as of mid-2024, US region).
    - This is billed as long as the model is deployed, even if you’re not making predictions.

- Model storage (GCS):
    - Typically a few cents per GB per month.

- Prediction requests:
    - Additional cost per 1,000 predictions (usually a few cents, depending on model type and region).

In [10]:
# Undeploy all models from the endpoint
# endpoint.undeploy_all()s

## Test Online Prediction

Now you can send requests to your endpoint using the Vertex AI SDK or REST API. Example code for sending a test request is available in the [Vertex AI docs](https://cloud.google.com/vertex-ai/docs/predictions/get-online-predictions).

**Tip:** For batch prediction, see [Vertex AI batch prediction docs](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions).